In [1]:
### Benjamin Tollison ###

from decimal import Decimal

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import sympy as sp
from IPython.display import Latex, Math, display
from sympy import (
    Eq,
    Function,
    Matrix,
    cos,
    cosh,
    exp,
    integrate,
    lambdify,
    pi,
    sin,
    sinh,
    symbols,
)
from sympy.solvers.pde import pdsolve
from sympy.solvers.solveset import linsolve


def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
def displayVariableWithUnits(variable:str,RHS,units):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    latexUnit = sp.latex(symbols(units))
    display(Math(left + '=' + right + '\\;' +'\\left['+ latexUnit + '\\right]'))
def format_scientific(number:float):
    a = '%E' % number
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]
deg2rad = np.pi/180
rad2deg = 180/np.pi

In [2]:
def PrandtlMeyerRadians(machnumber:float)->float:
  M = machnumber
  v = np.sqrt(2.4/.4)*np.arctan2(np.sqrt(.4*(M**2-1)),np.sqrt(2.4))-np.arctan2(np.sqrt(M**2-1),1)
  return v
def FindMachTwo(deflection_angle_degrees,mach_1):
  theta = deflection_angle_degrees*deg2rad
  nu2 = theta + PrandtlMeyerRadians(mach_1)
  M,gamma = symbols('M \\gamma')
  PrandtlMeyerSymbolic = sp.sqrt((gamma+1)/(gamma-1))*sp.atan(sp.sqrt((gamma-1)*(M**2-1)/(gamma+1)))-sp.atan(M**2-1)
  partial_M_PradtlMeyerSymbolic = PrandtlMeyerSymbolic.diff(M)
  step_function = lambdify(M,(PrandtlMeyerSymbolic/partial_M_PradtlMeyerSymbolic).subs({gamma:1.4}))
  display(step_function)
  f = lambdify(M,PrandtlMeyerSymbolic.subs({gamma:1.4}))
  M2_guess = 1.1
  iteration_stopper = 1000
  while abs(f(M2_guess)-nu2) > 1e-6 or abs(step_function(M2_guess)) > 1e-8:
    M2_guess = M2_guess - step_function(M2_guess)
    iteration_stopper -= 1
    if iteration_stopper == 0:
      print('Newton-Raphson did not converge')
      break
  displayVariable('M_2',M2_guess)
  return M2_guess

In [3]:
### Question 1
def PrandtlMeyerRadians(machnumber:float)->float:
  M = machnumber
  v = np.sqrt(2.4/.4)*np.arctan2(np.sqrt(.4*(M**2-1)),np.sqrt(2.4))-np.arctan2(np.sqrt(M**2-1),1)
  return v
displayVariable('\\nu(M_\\infty)',PrandtlMeyerRadians(np.inf)*rad2deg)

<IPython.core.display.Math object>

In [4]:
### Question 2
nu_m1 = 130.45-43
print(nu_m1)
FindMachTwo(43,6)

87.44999999999999


<function _lambdifygenerated(M)>

<lambdifygenerated-2>:2: RuntimeWarning: invalid value encountered in sqrt
  return 2.44948974278318*arctan(0.408248290463863*sqrt(M**2 - 1)) - arctan(M**2 - 1)
<lambdifygenerated-1>:2: RuntimeWarning: invalid value encountered in sqrt
  return (2.44948974278318*arctan(0.408248290463863*sqrt(M**2 - 1)) - arctan(M**2 - 1))/(-2*M/((M**2 - 1)**2 + 1) + 1.0*M/((0.166666666666667*M**2 + 0.833333333333333)*sqrt(M**2 - 1)))


<IPython.core.display.Math object>

-0.059713005952813925

In [5]:
### Question 4
def ExpansionPressureRatio(Mach1,Mach2):
  gamma = 1.4
  numerator = 1+((gamma-1)/2)*Mach1**2
  denominator = 1+((gamma-1)/2)*Mach2**2
  return numerator/denominator
M1 = 2
alpha = 15*deg2rad
p1 = 0.5 # bar
M1n = M1*np.cos(alpha)
displayVariable('M_{1\\,n}',M1n)
p3 = 4.179*p1
displayVariableWithUnits('P_3',p3,'bar')
nu_m2 = alpha + PrandtlMeyerRadians(2)
displayVariableWithUnits('\\nu(M_{2})',nu_m2*rad2deg,'deg')
p2 = p1*ExpansionPressureRatio(2,2.6)
displayVariableWithUnits('P_2',p2,'bar')
cord_length = 1
lift_per_unit_span = (p3-p2)*cord_length*np.cos(alpha)
displayVariable('L\'',lift_per_unit_span)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
### Question 5
drag_per_unit_span = (p3-p2)*cord_length*np.sin(alpha)
displayVariable('D\'',drag_per_unit_span)

<IPython.core.display.Math object>

In [7]:
### Question 6
displayVariableWithUnits('\\beta',np.arcsin(1/10)*rad2deg,'deg')

<IPython.core.display.Math object>

In [8]:
### Question 7
displayVariableWithUnits('\\alpha_{\\theta-\\beta-M}',44-25,'deg')
displayVariableWithUnits('\\alpha_\\nu',PrandtlMeyerRadians(np.inf)*rad2deg-102.3,'deg')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [9]:
### Question 9
displayVariable('\\frac{P_2}{P_1}',ExpansionPressureRatio(10,16))

<IPython.core.display.Math object>

In [10]:
### Question 10
M1 = 10
M1n = M1*np.cos(10*deg2rad)
displayVariable('M_{1\\,n}',M1n)
M3n = (M1n-9)*(.3876-.3898) + .3898
displayVariable('M_{3\\,n}',M3n)
Mtangental = M1*np.sin(10*deg2rad)
displayVariable('\\omega',Mtangental)
M3 = np.sqrt(M3n**2 + Mtangental**2)
displayVariable('M_3',M3)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [11]:
### Question 11
shock_pressure_ratio = (M1n-9)*(116.5-94.33) + 94.33
displayVariable('\\frac{P_3}{P_1}',shock_pressure_ratio)

<IPython.core.display.Math object>

In [12]:
### Question 12
nu_4 = (30+25)*deg2rad + PrandtlMeyerRadians(M3)
displayVariable('\\nu(M_{4})',nu_4*rad2deg)
M4 = 4.8
displayVariable('M_4',M4)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [13]:
### Question 13
p4_over_p1 = ExpansionPressureRatio(M3,M4)*shock_pressure_ratio
displayVariable('\\frac{P_4}{P_1}',p4_over_p1)

<IPython.core.display.Math object>

In [14]:
### Question 14 and 15
linear_coefficient_of_lift = (4*(10*deg2rad))/((10**2-1)**0.5)
linear_coefficient_of_drag = (4*((10*deg2rad)**2 + 1))/ (99)**0.5
displayVariable('C_l',linear_coefficient_of_lift)
displayVariable('C_d',linear_coefficient_of_drag)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [15]:
### Question 16
M1 = 10
nu_m2 = 10*deg2rad + PrandtlMeyerRadians(10)
displayVariable('\\nu(M_{2})',nu_m2*rad2deg)
M2 = 16
p2_over_p1 = ExpansionPressureRatio(10,16)
displayVariable('\\frac{P_2}{P_1}',p2_over_p1)
M1n = M1*np.cos((25+10)*deg2rad)
Mtangental = M1*np.sin(35*deg2rad)
displayVariable('M_{1\\,n}',M1n)
M3n = 0.3929
M3 = np.sqrt(M3n**2 + Mtangental**2)
displayVariable('M_3',M3)
displayVariable('\\nu(M_{4})',50+76.92)
M4 = 50
p4_over_p3 = ExpansionPressureRatio(M3,50)
displayVariable('\\frac{P_4}{P_3}',p4_over_p3)
C_l = (np.cos(10*deg2rad)/(1.4*10**2))*(74.5*(p4_over_p3+1)-2*p2_over_p1)
displayVariable('C_l',C_l)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
### Question 17
C_d = C_l*np.tan(10*deg2rad)
displayVariable('C_d',C_d)

<IPython.core.display.Math object>

In [17]:
nu_4 = (50)*deg2rad + PrandtlMeyerRadians(M3)
displayVariable('\\nu(M_{4})',nu_4*rad2deg)
M4 = 4.35
def LinearCp(angle,mach):
  cp = (2*(angle*deg2rad)) / (mach**2-1)**0.5
  return cp
Cpa = LinearCp(-10,10)
Cpb = LinearCp(35,10)
Cpc = LinearCp(-50,M4)
Cpu = Cpa
Cpl = Cpb+Cpc
print(Cpl-Cpu)

<IPython.core.display.Math object>

-0.25439555457088814
